# Creating simulations using `deepszsim`

This tutorial provides instructions on how to create galaxy cluster simulations with `deepszsim`. For a step-by-step guide to the functions included in the simulation, see `demo_full_pipeline.ipynb`

## Imports

In [1]:
from deepszsim import make_sz_cluster, dm_halo_dist, visualization, simclusters

import time
import h5py

## Creating simulations by hand

Create a new flat redshift and virial mass distribution using the function `flastdist_halo` from `dm_halo_dist` which uses random uniform generation. 

In [2]:
#Generate a new flat z, Mvir distribution and save to file: 
nsources=10 #Number of halos to generate
zdist,mdist=dm_halo_dist.flatdist_halo(0.01,1.1,1e14,2e15,nsources) #Generate a flat z, Mvir distribution for sims

Save our simulated data to a h5 file titled `massdist.h5`. 

In [3]:
sourceid=int(time.time()) #Create an initial ctime for the halo ID list to save catalog
idlist=[sourceid+x for x in range(len(zdist))] #Create the halo ID list for catalog

#Save this array to a h5 file
with h5py.File('massdist.h5', 'w') as data:
    data.create_dataset('Redshift', data=zdist)
    data.create_dataset('Mass', data=mdist)
    data.create_dataset('id', data=idlist)

We can now simulate submaps using our mass and redshift distribution using the `simulate_T_submaps` function from `make_sz_cluster`. This function returns an array of dicts with the following attributes (each dict contains the full information of each sim/cluster):
- M200
- R200
- redshift_z
- y_central
- ID
- cmb_map
- noise_map
- final_map

In [4]:
clusters = simclusters.simulate_clusters(mdist, zdist)

In [5]:
len(clusters.id_list)

10

## Creating simulations automatically

Alternately, we can do everything from soup to nuts with the `simulate_clusters` class

In [6]:
_ = clusters.get_T_maps()

/Users/sammcd/micromamba/envs/szsims/lib/python3.11/site-packages/deepszsim/filters.py:38: RuntimeWarning: Mean of empty slice.
  ring_mean = dT_map[(r >= radmax_pixels) & (r < radius_out_pixels)].mean()
/Users/sammcd/micromamba/envs/szsims/lib/python3.11/site-packages/numpy/core/_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


This class instance populates an attribute named `clusters`, which is a dictionary. This dictionary is indexed by a run ID, and each of its values is itself a dictionary, which in turn contains two more dictionaries: `params` and `maps`

In [7]:
k10 = list(clusters.clusters.keys())
k10

['0822711_021_556314',
 '0552296_070_281581',
 '1372133_084_806690',
 '1371424_008_408584',
 '1998618_075_449070',
 '0198589_065_786397',
 '1525903_108_312178',
 '1124602_018_031953',
 '1846569_059_568057',
 '0517993_041_352332']

In [8]:
clusters.clusters[k10[0]]['params'], clusters.clusters[k10[0]]['maps'].keys()

({'M200': 822711781329317.2,
  'redshift_z': 0.2176700775626845,
  'R200': 1.8181656839508509,
  'angsize_arcmin': 5.5262467021573425,
  'angsize500_arcmin': 5.5262467021573425,
  'image_size_pixels': 41,
  'dT_central': -242.36963689525484,
  'ap': -45.38114063656835},
 dict_keys(['conv_map', 'CMB_map', 'signal_map', 'beamsig_map', 'final_map']))

this class instance has a `save_map` method that can save individual maps as specified, or all clusters into a single nested file with `nest_h5 = True`, or all clusters individually

In [ ]:
# c10.save_map()